In [55]:
source_cfg = {"id": "WHO_PRE_EDU_PARTICIPATION", "file_format": "json"}

In [56]:
pwd

'/Users/mykhailoslukvin/repo/dv-data-pipeline'

In [57]:
cd ../../../../

/


/Users/mykhailoslukvin/.local/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [58]:
import io
import os
import json

import country_converter as coco
import pandas as pd

from dfpp.storage import StorageManager
from dfpp.storage.postgres import AsyncPGClient

cc = coco.CountryConverter()

In [59]:
async with StorageManager() as storage_manager:
    source_file_name = os.path.join(
        storage_manager.sources_path,
        f"{source_cfg['id'].upper()}.{source_cfg['file_format']}",
    )
    bytes_data = await storage_manager.read_blob(path=source_file_name)

source_data = bytes_data.decode("utf-8")
source_data_dict = json.loads(source_data)
df = pd.DataFrame(source_data_dict["value"])

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4324 entries, 0 to 4323
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  4324 non-null   int64  
 1   IndicatorCode       4324 non-null   object 
 2   SpatialDimType      4324 non-null   object 
 3   SpatialDim          4324 non-null   object 
 4   ParentLocationCode  4324 non-null   object 
 5   TimeDimType         4324 non-null   object 
 6   ParentLocation      4324 non-null   object 
 7   Dim1Type            4324 non-null   object 
 8   TimeDim             4324 non-null   int64  
 9   Dim1                4324 non-null   object 
 10  Dim2Type            0 non-null      object 
 11  Dim2                0 non-null      object 
 12  Dim3Type            0 non-null      object 
 13  Dim3                0 non-null      object 
 14  DataSourceDimType   0 non-null      object 
 15  DataSourceDim       0 non-null      object 
 16  Value 

In [61]:
df.head()

,Id,IndicatorCode,SpatialDimType,SpatialDim,ParentLocationCode,TimeDimType,ParentLocation,Dim1Type,TimeDim,Dim1,...,DataSourceDim,Value,NumericValue,Low,High,Comments,Date,TimeDimensionValue,TimeDimensionBegin,TimeDimensionEnd
0,4006230,SE_PRE_PARTN,COUNTRY,TJK,EUR,YEAR,Europe,SEX,2016,SEX_MLE,...,None,15,15.04271,None,None,None,2019-04-09T13:35:36+02:00,2016,2016-01-01T00:00:00+01:00,2016-12-31T00:00:00+01:00
1,2562434,SE_PRE_PARTN,COUNTRY,PRT,EUR,YEAR,Europe,SEX,2005,SEX_MLE,...,None,93.3,93.31432,None,None,None,2019-04-09T13:35:21+02:00,2005,2005-01-01T00:00:00+01:00,2005-12-31T00:00:00+01:00
2,4387417,SE_PRE_PARTN,COUNTRY,SVN,EUR,YEAR,Europe,SEX,2008,SEX_FMLE,...,None,89.1,89.07066,None,None,None,2019-04-09T13:35:31+02:00,2008,2008-01-01T00:00:00+01:00,2008-12-31T00:00:00+01:00
3,2943578,SE_PRE_PARTN,COUNTRY,STP,AFR,YEAR,Africa,SEX,2013,SEX_FMLE,...,None,49.4,49.41758,None,None,None,2019-04-09T13:35:28+02:00,2013,2013-01-01T00:00:00+01:00,2013-12-31T00:00:00+01:00
4,1406094,SE_PRE_PARTN,COUNTRY,LUX,EUR,YEAR,Europe,SEX,2001,SEX_BTSX,...,None,95.1,95.14964,None,None,None,2019-04-09T13:35:00+02:00,2001,2001-01-01T00:00:00+01:00,2001-12-31T00:00:00+01:00


In [63]:
df.columns

Index(['Id', 'IndicatorCode', 'SpatialDimType', 'SpatialDim',
       'ParentLocationCode', 'TimeDimType', 'ParentLocation', 'Dim1Type',
       'TimeDim', 'Dim1', 'Dim2Type', 'Dim2', 'Dim3Type', 'Dim3',
       'DataSourceDimType', 'DataSourceDim', 'Value', 'NumericValue', 'Low',
       'High', 'Comments', 'Date', 'TimeDimensionValue', 'TimeDimensionBegin',
       'TimeDimensionEnd'],
      dtype='object')

In [64]:
df["TimeDimType"].value_counts(dropna=False)

TimeDimType
YEAR    4324
Name: count, dtype: int64

In [65]:
assert sum(df["TimeDimType"] != "YEAR") == 0

In [66]:
df["TimeDimensionValue"].value_counts(dropna=False)

TimeDimensionValue
2015    333
2016    325
2014    312
2013    296
2012    272
2011    264
2010    253
2009    240
2007    227
2008    227
2006    214
2005    209
2017    208
2004    196
2001    189
2002    184
2003    183
2000    180
2018     12
Name: count, dtype: int64

In [67]:
df["IndicatorCode"].value_counts(dropna=False).shape[0]

1

In [68]:
assert df["IndicatorCode"].value_counts(dropna=False).shape[0] == 1

In [69]:
df["SpatialDimType"].value_counts()

SpatialDimType
COUNTRY    4324
Name: count, dtype: int64

In [70]:
df_source = df.copy()

In [71]:
df_source = df_source[df_source["SpatialDimType"] == "COUNTRY"].reset_index()